In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.tools import WikipediaQueryRun, YouTubeSearchTool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults


In [ ]:
load_dotenv()

In [ ]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os. getenv("LANGCHAIN_PROJECT")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os. environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os. environ["GROQ_API_KEY"]= GROQ_API_KEY
os. environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os. environ["LANGCHAIN_ENDPOINT"] = "https://api.smith. langchain.com"
os. environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [ ]:
llm = ChatGroq(model_name="Gemma2-9b-It")

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
llm.invoke("Hi")

### Predefine Tools

In [ ]:
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
print(tool.run({"query":"langchain"}))

In [ ]:
tool_2 = YouTubeSearchTool()
tool_2.run("freecodecamp")

In [ ]:
tool_3 = TavilySearchResults()
tool_3.invoke({"query": "Who is Chris Heria?"})

### Creating a Custom Tool

In [2]:
from langchain.agents import tool

@tool
def get_word_length(word:str) -> int:
    """ Return the length of a word. """

    return len(word)

In [3]:
get_word_length("abc")

C:\Users\hamid\AppData\Local\Temp\ipykernel_8488\1732944327.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_word_length("abc")


3

# Concept of Agents

In [ ]:
from langchain.agents import AgentType, load_tools, initialize_agent


In [ ]:
tool_4 = load_tools(["wikipedia"], llm=llm)

### This Agent Class is from the first version of LangChain (legacy)

In [ ]:
tool_5 = load_tools(["wikipedia"], llm=llm)

In [ ]:
agent = initialize_agent(tools=tool_5, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("What is current GDP of USA?")

In [ ]:
search = TavilySearchResults()

In [ ]:
search.invoke("What is the weather in SF?")

In [ ]:
tools = [search]

In [ ]:
from langchain import hub

# Get the prompt to use - you can modify this
prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
from langchain.agents import create_tool_calling_agent

agent_1 = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent_1, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Hi"})

In [ ]:
agent_executor.invoke({"input": "How is the weather in SF?"})

# implementing simple RAG

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
loader = WebBaseLoader("https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/")

docs = loader.load()

In [ ]:
docs

In [ ]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [ ]:
vector = FAISS.from_documents(documents=documents, embedding=embeddings)

retriever = vector.as_retriever()

In [ ]:
retriever.invoke("What is langgraph studio?")[0]

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever,
    "langgraph_studio",
    "Search for information about LangGraph. for any questions about LangGraph, you must use this tool!"
)

In [ ]:
tool_6 = [search, retriever_tool]

In [ ]:
from langchain.agents import create_tool_calling_agent

agent_2 = create_tool_calling_agent(llm=llm, tools=tool_6, prompt=prompt)

In [ ]:
agent_executor_1 = AgentExecutor(agent=agent_2, tools=tool_6, verbose=True)

In [ ]:
agent_executor_1.invoke({"input": "What is LangGraph studio and how to use it?"})

In [ ]:
agent_executor_1.invoke({"input": "What is the weather in SF?"})

# React Agent (Reasoning + Acting)

In [ ]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain.agents import Tool, create_react_agent

In [ ]:
google_search = GoogleSerperAPIWrapper()

tools_1 = [
    Tool(
        name="Web Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [3]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the finel answer to the original input question
Begin!
Question: {input}
Thought: {agent_scratchpad}'''

In [4]:
prompt_1 = PromptTemplate.from_template(template=template)

In [ ]:
search_agent = create_react_agent(llm=llm, tools=tools_1, prompt=prompt_1)

In [ ]:
agent_executor_2 = AgentExecutor(
    agent=search_agent,
    tools=tools_1,
    verbose=True,
    return_intermediate_steps=True
)

In [ ]:
agent_executor_2.invoke({"input": "Where is the hometown of the 2007 T20 Cricket world cup caption winner and his score?"})

In [ ]:
agent_executor_2.invoke({"input": "Who is the father of AI?"})

### ReAct with Custom Tool

In [ ]:
# Custom tool for the Agent
@tool
def get_employee_id(name):

    """To get employee id, it takes employee name as arguments
    name(str): Name of the employee"""

    fake_employees = {
        "Alice": "E001",
        "Bob": "E002",
        "Charlie": "E003",
        "Diana": "E004",
        "Evan": "E005",
        "Fiona": "E006",
        "George": "E007",
        "Hannah": "E008",
        "Ian": "E009",
        "Jasmine": "E010"}

    return fake_employees.get(name, "Employee not found")

In [ ]:
# Custom tool for the Agent
@tool
def get_employee_salary(employee_id):

    """To get the salary of an employee, it takes employee_id as input and return salary
    """
    employee_salaries = {
        "E001": 56000,
        "E002": 47000,
        "E003": 52000,
        "E004": 61000,
        "E005": 45000,
        "E006": 58000,
        "E007": 49000,
        "E008": 53000,
        "E009": 50000,
        "E010": 55000
    }
    return employee_salaries.get(employee_id, "Employee not found")

I

In [ ]:
# Saved React prompt in langchain hub, we could manually type the prompt as well.
prompt_2 = hub.pull("hwchase17/react")

In [ ]:
print(prompt_2.template)

In [ ]:
tools_list = [get_employee_salary, get_employee_id]

In [ ]:
agent_for_employee = create_react_agent(llm=llm, tools=tools_list, prompt=prompt_2)

In [ ]:
employee_agent_executor = AgentExecutor(agent=agent_for_employee, tools=tools_list, verbose=True)

In [ ]:
agent_for_employee.invoke({"input": "what is the Salary of Evan?"})

# Self-Ask with Search Agetn

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer


In [ ]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

model = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [ ]:
tools_for_gemini = [
    Tool(
        name="Intermediate Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [ ]:
# get the prompt to use - you can modify this

prompt_for_gemini = hub.pull("hwchase17/self-ask-with-search")

In [ ]:
print(prompt_for_gemini.temlate)

In [ ]:
# Create the self ask with search agent

self_agent = create_self_ask_with_search_agent(llm=model, tools=tools_for_gemini, prompt=prompt_for_gemini)

In [ ]:
# create an agent executor by passing in the agent and tools

agent_executor_for_gemini = AgentExecutor(agent=self_agent, tools=tools_for_gemini, verbose=True, handle_parsing_errors=True)

In [ ]:
agent_executor_for_gemini.invoke(
    {"input": "Where is a the biggest AI company?"}
)

In [ ]:
agent_executor_for_gemini.invoke(
    {"input": "Which are the most active companies in the world?"}
)